In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

In [3]:
def find_third_largest(nums):
    if len(nums) < 3:
        return "列表长度不足3"
    
    # 将列表去重并排序
    unique_nums = sorted(set(nums), reverse=True)
    
    if len(unique_nums) < 3:
        return "列表中没有第三大的数字"
    
    return unique_nums[2]

In [4]:
RAData = pd.read_csv("RA medication class(filtered).csv")
T2DData = pd.read_csv("T2D medication class(filtered).csv")
RAData = RAData.drop_duplicates(subset=["MedicationCui","ClassCui"]).reset_index(drop=True)
T2DData = T2DData.drop_duplicates(subset=["MedicationCui","ClassCui"]).reset_index(drop=True)

In [5]:
T2DData

,MedicationCui,MedicationStr,ClassCui,ClassStr,ClassCui(Synonym),ClassStr(Synonym)
0,C0008287,chlorpropamide,C0020616,Hypoglycemic Agents,C0017687,glukagon
1,C0008287,chlorpropamide,C0038766,Sulfonylurea Compounds,C0061323,glimepiride
2,C0017628,glyburide,C0017628,glyburide,C0081232,5-iodo-2-hydroxyglyburide
3,C0017628,glyburide,C0020616,Hypoglycemic Agents,C0017687,glukagon
4,C0017628,glyburide,C0038766,Sulfonylurea Compounds,C0061323,glimepiride
...,...,...,...,...,...,...
125,C3491971,insulin degludec,C0013227,Pharmaceutical Preparations,C0304232,Type of drug preparation
126,C3491971,insulin degludec,C0021641,Insulin,C0020616,hypoglykemika
127,C3555622,dapagliflozin propanediol,C2353951,dapagliflozin,NaN,NaN
128,C3555622,dapagliflozin propanediol,C3273807,Sodium-Glucose Transporter 2 Inhibitors,NaN,NaN


In [6]:
## Change T2DData[['MedicationStr','ClassStr']] into Dictionary. The key is ClassStr, and the value is MedicationStr which belongs to the ClassStr.
T2DDataDict = {}
for i in range(len(T2DData)):
    if T2DData.loc[i,'ClassStr'] not in T2DDataDict:
        T2DDataDict[T2DData.loc[i,'ClassStr']] = [T2DData.loc[i,'MedicationStr']]
    else:
        T2DDataDict[T2DData.loc[i,'ClassStr']].append(T2DData.loc[i,'MedicationStr'])

In [7]:
## Intersection Number
IntersectionNumber = {}
for key1 in T2DDataDict.keys():
    IntersectionNumber[key1] = 0
    for key2 in T2DDataDict.keys():
        if key1 != key2 and len(set(T2DDataDict[key1]) & set(T2DDataDict[key2]))>0:
            IntersectionNumber[key1] += 1

In [8]:
IntersectionNumber

{'Hypoglycemic Agents': 28,
 'Sulfonylurea Compounds': 4,
 'glyburide': 2,
 'glipizide': 2,
 'Carbohydrates': 7,
 'acarbose': 7,
 'Meglitinide': 9,
 'Oral hypoglycemic': 13,
 'Antidiabetics': 19,
 'alpha-Glucosidase Inhibitors': 7,
 'Glucosidase Inhibitor': 7,
 'glimepiride': 2,
 'Thiazolidinediones': 4,
 'Insulin': 6,
 'Insulin, Short-Acting': 3,
 'insulin, regular, human': 7,
 'exenatide': 4,
 'Product containing glucagon-like peptide 1 receptor agonist (product)': 6,
 'Incretins': 6,
 'Glinides': 4,
 'Pharmaceutical Preparations': 4,
 'Quaternary Ammonium Compounds': 1,
 'metformin': 1,
 'insulin isophane': 3,
 'insulin isophane, human': 3,
 'pioglitazone': 1,
 'nateglinide': 4,
 'Insulin, Long-Acting': 4,
 'Basal insulin': 4,
 'Dipeptidyl-Peptidase IV Inhibitors': 3,
 'dapagliflozin': 3,
 'Sodium-Glucose Transporter 2 Inhibitors': 5,
 'Sodium glucose cotransporter subtype 2 inhibitor': 5,
 'canagliflozin': 3,
 'Peptides': 4,
 'dulaglutide': 4,
 'empagliflozin': 3}

In [9]:
## find the class which has the most intersection with other classes
class_to_remove = []

threshold = find_third_largest(list(IntersectionNumber.values()))
for key in IntersectionNumber.keys():
    if IntersectionNumber[key] >= threshold:
        class_to_remove.append(key)
        
for key in class_to_remove:
    del T2DDataDict[key]

In [10]:
## remove the key which is the subset of other key
class_to_remove = []  
class_used = []
for key in T2DDataDict.keys():
    data = T2DDataDict[key]
    for supkey in T2DDataDict.keys():
        if supkey == key or supkey in class_used:
            continue
        supdata = T2DDataDict[supkey]
        if set(data).issubset(set(supdata)):
            class_used.append(key)
            class_to_remove.append(key)  

class_to_remove = list(set(class_to_remove))
class_to_remove

['exenatide',
 'Basal insulin',
 'Peptides',
 'alpha-Glucosidase Inhibitors',
 'insulin isophane, human',
 'glimepiride',
 'dapagliflozin',
 'nateglinide',
 'glipizide',
 'glyburide',
 'Glinides',
 'canagliflozin',
 'dulaglutide',
 'empagliflozin',
 'Quaternary Ammonium Compounds',
 'Sodium glucose cotransporter subtype 2 inhibitor',
 'Insulin, Short-Acting',
 'insulin isophane',
 'Carbohydrates',
 'pioglitazone',
 'Insulin, Long-Acting',
 'acarbose']

In [11]:
for key in class_to_remove:
    del T2DDataDict[key]

In [12]:
#Judge the number of the class contain one medication
single_class = []
for key in T2DDataDict.keys():
    if len(T2DDataDict[key]) == 1:
        single_class.append(key)
#Change the key of the single class to 'other' 
if len(single_class) > 0:
    T2DDataDict['other'] = T2DDataDict[single_class[0]]
    del T2DDataDict[single_class[0]]
    for i in range(1,len(single_class)):
        T2DDataDict['other'] = T2DDataDict['other'] + T2DDataDict[single_class[i]]
        del T2DDataDict[single_class[i]]

In [13]:
#change the dictionary into dataframe
T2DData = pd.DataFrame(columns=['ClassStr', 'MedicationStr'])
for key in T2DDataDict.keys():
    for value in T2DDataDict[key]:
        row = {'ClassStr': key, 'MedicationStr': value}
        T2DData = pd.concat([T2DData, pd.DataFrame([row])])

In [14]:
T2DData['Disease'] = 'T2D'

In [15]:
T2DData.to_csv("PlotMedicationData(T2D).csv",index=False,encoding='utf_8_sig')

## ALL

In [16]:
fig = px.sunburst(T2DData,
                  path=["Disease", "ClassStr", "MedicationStr",],
                  width=750, height=750,
                  title="T2D Medication(Rule Filtered)",
                  )
fig.write_html("T2D Medication(Rule Filtered).html")
fig.show()